In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Lambda
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
filename = "wonderland.txt"
all_text = open("data/shakespeare.txt").read().lower()
all_text = ''.join([i for i in all_text if not i.isdigit()])
SYMBOL_LIST = {'?', '.',  ')', ',', '!', ';', '(', ':'}
sonnets = all_text.replace('                   ', '').split('\n\n\n')
sonnets = [sonnet[1:] for sonnet in sonnets]
sonnets = [sonnet.replace('\n', ' ') for sonnet in sonnets]
sonnets = [sonnet.replace('  ', '') for sonnet in sonnets]
sonnets = [''.join([i for i in sonnet if not i in SYMBOL_LIST]) for sonnet in sonnets]

all_chars = sorted(list(set(''.join(sonnets))))
max_char = len(all_chars) - 1
char_map = {}
int_to_char = {}
int_to_arr = {}
for i in range(len(all_chars)):
    char_map[all_chars[i]] = i
    int_to_char[i] = all_chars[i]
    arr = [0] * len(all_chars) 
    arr[i] = 1
    int_to_arr[i] = arr

# print(sonnets)

X_train = []
Y_train = []

for sonnet in sonnets:
    start = 0
    end = 40
    while end < len(sonnet):
        sequence = sonnet[start:end]
        X_train.append([(char_map[c]) for c in sequence])
        Y_train.append(char_map[sonnet[end]])
        start += 1
        end += 1
        
samples = len(X_train)
timesteps = 40
features = len(all_chars)
X_train = [[int_to_arr[x] for x in seq] for seq in X_train]

X_train = np.array(X_train) 
X_train = np.reshape(X_train, (samples, timesteps, features))
Y_train = np.array([int_to_arr[y] for y in Y_train])


print(samples)

84692


In [ ]:
# LSTM Layers:
temp = 0.75

model = Sequential()
model.add(LSTM(200, input_shape=(timesteps, features), return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(150))
model.add(Dropout(0.1))
model.add(Lambda(lambda x: x / temp))
model.add(Dense(Y_train.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(X_train, Y_train, epochs=50, batch_size=128)


    

In [ ]:
def next_char(input_str, m):
    X_in = np.array([[int_to_arr[char_map[c]] for c in input_str]])
    [probs] = m.predict(X_in, verbose=0)
    options = list(range(len(probs)))

    
    prediction = np.random.choice(options,p =  probs)
    
    return int_to_char[prediction]

result = "shall i compare thee to a summer's day t"
window = "shall i compare thee to a summer's day t"
for i in range(1000):
    prediction = next_char(window, model)
    result = result + prediction
    window = window[1:] + prediction

print(result)

In [ ]:
def get_poem(t, words_per_line, num_lines):
    result = []

    start = 0
    end = -1
    space_count = 0

    for i in range(len(t)):
        end += 1
        if t[i] == ' ':
            space_count += 1
        if space_count > words_per_line - 1:
            result.append(t[start:end])
            start = end + 1
            space_count = 0
        if i == len(t) - 1:
            result.append(t[start: end + 1])
        if len(result) == num_lines:
            break
    return result

In [ ]:
poem = get_poem(result, 8, 10)
for i in range(len(poem)):
    if i % 2 == 0:
        print(poem[i] + ',')
    else:
        print(poem[i] + '.')